In [5]:


# === Set your API keys ===
api_key = '0x4782BB28913a74A89a3625c88058C911d0b675Be'
api_secret = '0x922cf5042c5ac80c3df1519bbdb045f53b8e8b0c9de41619bc8ae422f2e347e5'

import ccxt
import csv
import time
import os


exchange_options = {
    'walletAddress': api_key,
    'enableRateLimit': True,
    'privateKey': api_secret,
    'options': {
        'adjustForTimeDifference': True,
        'recvWindow': 10000,
        'user': api_key,
        'defaultSlippage': 0.01,   # 1%
    }
}

# ─── 2) Instantiate Hyperliquid client ───────────────────────────────────────────
exchange = ccxt.hyperliquid(exchange_options)

# ─── 3) Load all markets & symbols ───────────────────────────────────────────────
exchange.load_markets()
all_symbols = exchange.symbols

# ─── 4) Prepare CSV output ──────────────────────────────────────────────────────
output_file = 'hyperliquid_all_trades.csv'
fieldnames = [
    'symbol',
    'trade_id',
    'order_id',
    'timestamp',
    'datetime',
    'side',
    'price',
    'amount',
    'cost',
    'fee'
]

with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # ─── 5) For each symbol, page through your entire history ────────────────────
    for symbol in all_symbols:
        print(f"📡 Fetching trades for {symbol}…")
        since = None
        limit = 1000   # adjust if Hyperliquid supports a different max
        all_trades = []

        while True:
            try:
                trades = exchange.fetch_my_trades(symbol, since, limit)
            except ccxt.BadSymbol:
                # you never traded this pair → skip
                break
            except Exception as e:
                print(f"   ⚠️  Error on {symbol}: {e}. Retrying in 5s…")
                time.sleep(5)
                continue

            if not trades:
                break

            all_trades.extend(trades)
            since = trades[-1]['timestamp'] + 1
            time.sleep(exchange.rateLimit / 1000)

        # ─── 6) Write out each trade ────────────────────────────────────────────
        for t in all_trades:
            writer.writerow({
                'symbol':    symbol,
                'trade_id':  t['id'],
                'order_id':  t.get('order'),
                'timestamp': t['timestamp'],
                'datetime':  t['datetime'],
                'side':      t['side'],
                'price':     t['price'],
                'amount':    t['amount'],
                'cost':      t.get('cost'),
                'fee':       f"{t['fee']['cost']} {t['fee']['currency']}" if t.get('fee') else None,
            })

print(f"\n✅ All Hyperliquid trades saved to {output_file}")


📡 Fetching trades for AAVE/USDC:USDC…
📡 Fetching trades for ACE/USDC:USDC…
📡 Fetching trades for ADA/USDC:USDC…
📡 Fetching trades for ADHD/USDC…
📡 Fetching trades for AI/USDC:USDC…
📡 Fetching trades for AI16Z/USDC:USDC…
📡 Fetching trades for AIXBT/USDC:USDC…
📡 Fetching trades for ALGO/USDC:USDC…
📡 Fetching trades for ALT/USDC:USDC…
📡 Fetching trades for ANIME/USDC:USDC…
📡 Fetching trades for ANON/USDC…
📡 Fetching trades for ANSEM/USDC…
📡 Fetching trades for ANT/USDC…
📡 Fetching trades for APE/USDC:USDC…
📡 Fetching trades for APT/USDC:USDC…
📡 Fetching trades for AR/USDC:USDC…
📡 Fetching trades for ARB/USDC:USDC…
📡 Fetching trades for ARI/USDC…
📡 Fetching trades for ARK/USDC:USDC…
📡 Fetching trades for ASI/USDC…
📡 Fetching trades for ATEHUN/USDC…
📡 Fetching trades for ATOM/USDC:USDC…
📡 Fetching trades for AUTIST/USDC…
📡 Fetching trades for AVAX/USDC:USDC…
📡 Fetching trades for BABY/USDC:USDC…
📡 Fetching trades for BADGER/USDC:USDC…
📡 Fetching trades for BAGS/USDC…
📡 Fetching trades for B

In [11]:
import pandas as pd
file = pd.read_csv(output_file)  # Display the first few rows of the CSV file
file = file.sort_values(by='timestamp')  # Sort by timestamp
file.to_csv(output_file, index=False)  # Save the sorted DataFrame back to CSV